matching of features in the train and test data is essential.
prior probability joint and class probabilities are calculate in Naive-Bayes theorem

In [5]:
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from itertools import islice
from sklearn.naive_bayes import MultinomialNB
lemm =WordNetLemmatizer()

#import pickle


In [9]:
df=pd.read_csv("C:\\Users\\SAIRAM YERRAMSETTI\\Documents\\data sets\\capstone nlp\\News\\News\\train.csv",header= None)

In [10]:
df.head()

,0,1,2
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [11]:
df.shape

(120000, 3)

In [13]:
df.columns=["Class","ShortDesc","Desc"]
df.head()

,Class,ShortDesc,Desc
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [19]:
##cleansing steps which are used on both train and test:

nltk.download('stopwords')
stops= stopwords.words('english')


[nltk_data] Downloading package stopwords to C:\Users\SAIRAM
[nltk_data]     YERRAMSETTI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [22]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\SAIRAM
[nltk_data]     YERRAMSETTI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [34]:
#import nltk
def _remove_noise(input_text):
    input_text = str(input_text).encode('ascii','ignore')
    input_text = str(input_text).replace(",","")
    input_text = str(input_text).replace("\'","")
    input_text = str(input_text).replace("\\","")
    input_text = str(input_text).replace("-","")
    input_text = str(input_text).replace(".","")
    input_text = re.sub("\d+","",input_text)
    input_text = re.sub("[^a-zA-Z0-9 ]","",input_text)
    words = str(input_text).split()
    noise_free_words = [word for word in words if word.lower() not in stops]
    lower_words = [word.lower() for word in noise_free_words]
    lower_words = [lemm.lemmatize(w) for w in lower_words]
   
   
    return lower_words

In [35]:
text = " This is a Test to remove numbers\. 1234., and special characters"

In [36]:
_remove_noise(text)

['b', 'test', 'remove', 'number', 'special', 'character']

In [38]:
a = _remove_noise(text)

In [40]:
a[0]
   

'b'

In [41]:
df["cleaned"]= df.Desc.apply(_remove_noise)
df.head()

,Class,ShortDesc,Desc,cleaned
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...","[breuters, shortsellers, wall, street, dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,"[breuters, private, investment, firm, carlyle,..."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,"[breuters, soaring, crude, price, plus, worrie..."
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,"[breuters, authority, halted, oil, exportflows..."
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","[bafp, tearaway, world, oil, price, toppling, ..."


In [44]:
#conversion of list into string
#from nltk.stem import WordNetLemmatizer
df['Lemmed'] = df.cleaned.map(lambda x: ' '.join([ y for y in x ]))
df.head()

,Class,ShortDesc,Desc,cleaned,Lemmed
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...","[breuters, shortsellers, wall, street, dwindli...",breuters shortsellers wall street dwindlingban...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,"[breuters, private, investment, firm, carlyle,...",breuters private investment firm carlyle group...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,"[breuters, soaring, crude, price, plus, worrie...",breuters soaring crude price plus worriesabout...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,"[breuters, authority, halted, oil, exportflows...",breuters authority halted oil exportflows main...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","[bafp, tearaway, world, oil, price, toppling, ...",bafp tearaway world oil price toppling record ...


In [52]:
#intilisation
cvec= CountVectorizer(stop_words=stops, min_df= 0.001, max_df=0.7, ngram_range=(1,3))
cvec.fit(df.Lemmed)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.7, max_features=None, min_df=0.001,
                ngram_range=(1, 3), preprocessor=None,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [51]:
len(cvec.vocabulary_)

3732

In [89]:
cvec_counts = cvec.transform(df.Lemmed)



In [90]:
print ('Sparse matrix  shape:', cvec_counts.shape)
print ('Non zero count:', cvec_counts.nnz)

Sparse matrix  shape: (120000, 3732)
Non zero count: 1875589


In [91]:
print ('sparsity: %.2f%%' %(100.0 * cvec_counts.nnz / (cvec_counts.shape[0]*cvec_counts.shape[1])))

sparsity: 0.42%


In [59]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<120000x3732 sparse matrix of type '<class 'numpy.float64'>'
	with 1875589 stored elements in Compressed Sparse Row format>

In [61]:
transformed_weights.shape

(120000, 3732)

In [63]:
target = df["Class"]

In [67]:
nb = MultinomialNB()

In [68]:
nb.fit(transformed_weights, target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [69]:
df1 =pd.read_csv("C:\\Users\\SAIRAM YERRAMSETTI\\Documents\\data sets\\capstone nlp\\News\\News\\test.csv",header= None)

In [70]:
df1.columns=["Class","ShortDesc","Desc"]
df1.head()

,Class,ShortDesc,Desc
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [75]:
df1["cleaned"]= df1.Desc.apply(_remove_noise)
df1.head()

,Class,ShortDesc,Desc,cleaned,Lemmed
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,"[bunion, representing, worker, turner, newall,...",breuters shortsellers wall street dwindlingban...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o...","[bspacecom, toronto, canada, secondteam, rocke...",breuters private investment firm carlyle group...
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...,"[bap, company, founded, chemistry, researcher,...",breuters soaring crude price plus worriesabout...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...,"[bap, barely, dawn, mike, fitzpatrick, start, ...",breuters authority halted oil exportflows main...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...,"[bap, southern, california, smogfighting, agen...",bafp tearaway world oil price toppling record ...


In [76]:
df1['Lemmed'] = df1.cleaned.map(lambda x: ' '.join([ y for y in x ]))
df1.head()

,Class,ShortDesc,Desc,cleaned,Lemmed
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,"[bunion, representing, worker, turner, newall,...",bunion representing worker turner newall say d...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o...","[bspacecom, toronto, canada, secondteam, rocke...",bspacecom toronto canada secondteam rocketeers...
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...,"[bap, company, founded, chemistry, researcher,...",bap company founded chemistry researcher unive...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...,"[bap, barely, dawn, mike, fitzpatrick, start, ...",bap barely dawn mike fitzpatrick start shift b...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...,"[bap, southern, california, smogfighting, agen...",bap southern california smogfighting agency we...


In [92]:
cvec1= CountVectorizer(stop_words=stops, min_df= 0.001, max_df=0.7, ngram_range=(1,3))
cvec1.fit(df1.Lemmed)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.7, max_features=None, min_df=0.001,
                ngram_range=(1, 3), preprocessor=None,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [93]:
len(cvec1.vocabulary_)

3972

In [94]:
cvec1= CountVectorizer(stop_words=stops, min_df= 0.005, max_df=0.7)
cvec1.fit(df1.Lemmed)
len(cvec1.vocabulary_)

717

In [95]:
cvec_counts1= cvec.transform(df1.Lemmed)
print ('Sparse matrix  shape:', cvec_counts1.shape)
print ('Non zero count:', cvec_counts1.nnz)
print ('sparsity: %.2f%%' %(100.0 * cvec_counts1.nnz / (cvec_counts1.shape[0]*cvec_counts1.shape[1])))

Sparse matrix  shape: (7600, 3732)
Non zero count: 118004
sparsity: 0.42%


In [96]:
transformer1 = TfidfTransformer()
transformed_weights1 = transformer.fit_transform(cvec_counts1)
transformed_weights1

<7600x3732 sparse matrix of type '<class 'numpy.float64'>'
	with 118004 stored elements in Compressed Sparse Row format>

In [97]:
transformed_weights1.shape

(7600, 3732)

In [98]:
preds = nb.predict(transformed_weights1)

In [99]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [100]:
confusion_matrix(df1["Class"],preds)

array([[1666,   78,   95,   61],
       [  37, 1816,   17,   30],
       [  93,   25, 1537,  245],
       [  93,   54,  179, 1574]], dtype=int64)

In [101]:
accuracy_score(df1["Class"], preds, normalize=True)

0.8675